In [11]:
def forisek_jancina_hash(x: int) -> int:
    h = x & 0xFFFFFFFF
    h = (((h >> 16) ^ h) * 0x45D9F3B) & 0xFFFFFFFF
    h = (((h >> 16) ^ h) * 0x45D9F3B) & 0xFFFFFFFF
    return ((h >> 16) ^ h) & 0xFF

print("Hash(123456789) =", forisek_jancina_hash(123456789))


Hash(123456789) = 116


In [12]:
def is_SPRP(n: int, a: int) -> bool:
    if n < 2:
        return False
    if n & 1 == 0:
        return n == 2
    d, s = n - 1, 0
    while d & 1 == 0:
        d >>= 1
        s += 1
    x = pow(a, d, n)
    if x in (1, n - 1):
        return True
    for _ in range(s - 1):
        x = (x * x) % n
        if x == n - 1:
            return True
    return False


print("is_SPRP(101,2) =", is_SPRP(101, 2))

is_SPRP(101,2) = True


In [13]:
# globale Konstante als Tuple statt Liste
MR_BASES_32BIT = (2, 7, 61)


def fj32_is_prime(n: int) -> bool:
    if n < 2:
        return False
    for p in (2, 3, 5, 7):
        if n == p:
            return True
        if n % p == 0:
            return False
    h = forisek_jancina_hash(n)
    base = MR_BASES_32BIT[h % len(MR_BASES_32BIT)]
    return is_SPRP(n, base)


print([fj32_is_prime(n) for n in (101, 102, 103, 104)])

[True, False, True, False]


In [14]:
# Zelle 4 – Benchmark Forisek-Jančina vs. Standard-MR

import random
import time


# Standard-Miller-Rabin für 32-Bit mit Basen 2,7,61
def standard_mr_32(n: int) -> bool:
    for a in MR_BASES_32BIT:
        if not is_SPRP(n, a):
            return False
    return True


# Testdaten: Zufällige ungerade 32-Bit-Zahlen
TEST_COUNT = 100_000
data = [random.randrange(3, 2**32, 2) for _ in range(TEST_COUNT)]

# Warmup
_ = fj32_is_prime(101)
_ = standard_mr_32(101)

# Benchmark FJ-Test
start = time.perf_counter()
for n in data:
    fj32_is_prime(n)
fj_time = time.perf_counter() - start

# Benchmark Standard-MR
start = time.perf_counter()
for n in data:
    standard_mr_32(n)
mr_time = time.perf_counter() - start

print(f"FJ32-Test:   {TEST_COUNT/mr_time/1e6:.3f} M ops/s ({mr_time:.2f}s total)")
print(f"Standard-MR: {TEST_COUNT/mr_time/1e6:.3f} M ops/s ({mr_time:.2f}s total)")
print(f"Speedup: {mr_time/fj_time:.2f}×")

FJ32-Test:   0.055 M ops/s (1.81s total)
Standard-MR: 0.055 M ops/s (1.81s total)
Speedup: 1.96×


In [15]:
import numba as nb


@nb.njit(cache=True)
def modpow_nb(a: int, d: int, n: int) -> int:
    result = 1
    base = a % n
    exp = d
    while exp > 0:
        if exp & 1:
            result = (result * base) % n
        base = (base * base) % n
        exp >>= 1
    return result


@nb.njit(cache=True)
def hash_nb(x: int) -> int:
    h = x & 0xFFFFFFFF
    h = (((h >> 16) ^ h) * 0x45D9F3B) & 0xFFFFFFFF
    h = (((h >> 16) ^ h) * 0x45D9F3B) & 0xFFFFFFFF
    return ((h >> 16) ^ h) & 0xFF


@nb.njit(cache=True)
def sprp_nb(n: int, a: int) -> nb.boolean:
    if n < 2:
        return False
    if (n & 1) == 0:
        return n == 2
    d = n - 1
    s = 0
    while (d & 1) == 0:
        d >>= 1
        s += 1
    x = modpow_nb(a, d, n)
    if x == 1 or x == n - 1:
        return True
    for _ in range(s - 1):
        x = (x * x) % n
        if x == n - 1:
            return True
    return False


@nb.njit(cache=True)
def fj32_nb(n: int) -> nb.boolean:
    if n < 2:
        return False
    # kleine Faktoren
    if n % 2 == 0:
        return n == 2
    if n % 3 == 0:
        return n == 3
    if n % 5 == 0:
        return n == 5
    if n % 7 == 0:
        return n == 7
    h = hash_nb(n)
    base = MR_BASES_32BIT[h % len(MR_BASES_32BIT)]
    return sprp_nb(n, base)

In [16]:
# Benchmark Zelle
start = time.perf_counter()
for n in data:
    fj32_nb(n)
fj_nb_time = time.perf_counter() - start

start = time.perf_counter()
for n in data:
    standard_mr_32(n)
mr_time = time.perf_counter() - start

print(
    f"FJ32-Numba:   {TEST_COUNT/fj_nb_time/1e6:.3f} M ops/s ({fj_nb_time:.2f}s total)"
)
print(f"Standard-MR:  {TEST_COUNT/mr_time/1e6:.3f} M ops/s ({mr_time:.2f}s total)")
print(f"Speedup JIT:  {mr_time/fj_nb_time:.2f}×")

FJ32-Numba:   0.070 M ops/s (1.43s total)
Standard-MR:  0.068 M ops/s (1.48s total)
Speedup JIT:  1.03×


In [17]:
# Zelle 7 – Numba-Parallel-Benchmark
from numba import prange


@nb.njit(parallel=True, cache=True)
def fj32_nb_parallel(arr):
    count = 0
    for i in prange(len(arr)):
        if fj32_nb(arr[i]):
            count += 1
    return count


# Aufbau der Daten bleibt gleich
data = data  # bestehende Liste von zufälligen Zahlen

# Parallel-Warmup
_ = fj32_nb_parallel(data)

# Parallel-Benchmark
start = time.perf_counter()
count = fj32_nb_parallel(data)
fj_par_time = time.perf_counter() - start

print(
    f"FJ32-Numba-Parallel: {TEST_COUNT/fj_par_time/1e6:.3f} M ops/s ({fj_par_time:.2f}s total)"
)

FJ32-Numba-Parallel: 0.271 M ops/s (0.37s total)


In [18]:
# Zelle 8 – CFFI C-Extension Setup

import subprocess
import sys

# CFFI installieren (falls nicht vorhanden)
try:
    import cffi

    print("CFFI bereits installiert")
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "cffi"])
    import cffi

# C-Code für FJ32-Hash und SPRP
c_source = """
#include <stdint.h>

uint32_t fj_hash_c(uint32_t x) {
    uint32_t h = x;
    h = (((h >> 16) ^ h) * 0x45D9F3B) & 0xFFFFFFFF;
    h = (((h >> 16) ^ h) * 0x45D9F3B) & 0xFFFFFFFF;
    return ((h >> 16) ^ h) & 0xFF;
}

uint64_t modpow_c(uint64_t a, uint64_t d, uint64_t n) {
    uint64_t result = 1;
    uint64_t base = a % n;
    while (d > 0) {
        if (d & 1) {
            result = (__uint128_t)result * base % n;
        }
        base = (__uint128_t)base * base % n;
        d >>= 1;
    }
    return result;
}

int sprp_c(uint64_t n, uint64_t a) {
    if (n < 2) return 0;
    if ((n & 1) == 0) return n == 2;
    
    uint64_t d = n - 1, s = 0;
    while ((d & 1) == 0) { d >>= 1; s++; }
    
    uint64_t x = modpow_c(a, d, n);
    if (x == 1 || x == n - 1) return 1;
    
    for (uint64_t i = 1; i < s; i++) {
        x = (__uint128_t)x * x % n;
        if (x == n - 1) return 1;
    }
    return 0;
}

int fj32_c(uint32_t n) {
    if (n < 2) return 0;
    if (n % 2 == 0) return n == 2;
    if (n % 3 == 0) return n == 3;
    if (n % 5 == 0) return n == 5;
    if (n % 7 == 0) return n == 7;
    
    uint32_t bases[] = {2, 7, 61};
    uint32_t h = fj_hash_c(n);
    uint32_t base = bases[h % 3];
    return sprp_c(n, base);
}
"""

# CFFI Builder
ffibuilder = cffi.FFI()
ffibuilder.cdef(
    """
    uint32_t fj_hash_c(uint32_t x);
    int fj32_c(uint32_t n);
"""
)

ffibuilder.set_source("_fj32_c", c_source)
ffibuilder.compile(tmpdir=".")

print("C-Extension kompiliert")

CFFI bereits installiert


VerificationError: CompileError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2022\\BuildTools\\VC\\Tools\\MSVC\\14.44.35207\\bin\\HostX86\\x64\\cl.exe' failed with exit code 2

In [ ]:
import primetest

print("Hash C:", primetest.fj_hash_c(123456789))
print("FJ32 C:", primetest.fj32_c(101))

Hash C: 116
FJ32 C: True


In [19]:
import time
import random

# Testdaten neu generieren
TEST_COUNT = 100_000
data = [random.randrange(3, 2**32, 2) for _ in range(TEST_COUNT)]

# Numba-Benchmark erneut, um fj_nb_time zu definieren
start = time.perf_counter()
for n in data:
    fj32_nb(n)
fj_nb_time = time.perf_counter() - start

# C-Extension Benchmark
start = time.perf_counter()
for n in data:
    primetest.fj32_c(n)
c_time = time.perf_counter() - start

print(f"C-Extension: {TEST_COUNT/c_time/1e6:.3f} M ops/s ({c_time:.2f}s total)")
print(f"Numba:       {TEST_COUNT/fj_nb_time/1e6:.3f} M ops/s ({fj_nb_time:.2f}s total)")

C-Extension: 0.920 M ops/s (0.11s total)
Numba:       1.229 M ops/s (0.08s total)
